## Imports

In [1]:
import numpy as np
import plotly.graph_objects as go

# 1. The Math Engine
This function is the core of the notebook. It takes a covariance matrix and transforms a generic unit sphere into an ellipsoid that matches your data's distribution.

**Key Math Concepts:**

- **Eigenvalues:** Determine how long the ellipsoid is (the stretch).

- **Eigenvectors:** Determine the orientation of the ellipsoid (the rotation).


## 🧮 Mathematical Walkthrough: From Data to Ellipsoid

This section traces the transformation of **one single point** from the Unit Sphere to the final Ellipsoid using the data generated in the code above.

### 1. The Inputs
We start with the **Covariance Matrix** ($\Sigma$) and **Mean Vector** ($\mu$) defined in the code:

$$
\Sigma = \begin{bmatrix}
3.0 & 1.5 & 0.5 \\
1.5 & 2.0 & 0.2 \\
0.5 & 0.2 & 0.5
\end{bmatrix}
, \quad
\mu = \begin{bmatrix} 5 \\ 0 \\ 2 \end{bmatrix}
$$

### 2. Eigen Decomposition
The function `np.linalg.eigh(Sigma)` decomposes the matrix to find the principal axes.

**Eigenvalues ($\lambda$):**
These represent the variance ($\sigma^2$) along the principal axes.
$$
\lambda \approx [4.15, \quad 0.96, \quad 0.39]
$$

**Eigenvectors ($V$):**
These represent the rotation matrix. Columns are the normalized axis vectors.
$$
V \approx \begin{bmatrix}
0.78 & -0.58 & 0.23 \\
0.57 & 0.81 & 0.12 \\
0.25 & -0.05 & -0.96
\end{bmatrix}
$$

---

### 3. The Transformation Pipeline
To generate the ellipsoid surface, we transform a point $P_{sphere}$ (from a unit sphere) into $P_{final}$ using the following formula:

$$
P_{final} = \mu + (V \cdot S \cdot P_{sphere})
$$

Where $S$ is the **Scaling Matrix**.

#### Step A: Creating the Scaling Matrix ($S$)
We calculate $S$ using the square root of eigenvalues (to get standard deviation) multiplied by our confidence interval ($k=2.0$).

$$S_{ii} = \sqrt{\lambda_i} \cdot k$$

$$
S \approx \begin{bmatrix}
\sqrt{4.15} \cdot 2 & 0 & 0 \\
0 & \sqrt{0.96} \cdot 2 & 0 \\
0 & 0 & \sqrt{0.39} \cdot 2
\end{bmatrix}
\approx
\begin{bmatrix}
4.08 & 0 & 0 \\
0 & 1.96 & 0 \\
0 & 0 & 1.24
\end{bmatrix}
$$

#### Step B: Tracing a Single Point
Let's track a point $\mathbf{p}$ lying on the "North Pole" of the unit sphere: $\mathbf{p} = [0, 0, 1]^T$.

**1. Scaling (Stretch):**
$$
P_{scaled} = S \cdot \mathbf{p} = 
\begin{bmatrix} 4.08 & 0 & 0 \\ 0 & 1.96 & 0 \\ 0 & 0 & 1.24 \end{bmatrix} 
\begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} 
= \begin{bmatrix} 0 \\ 0 \\ 1.24 \end{bmatrix}
$$

**2. Rotation (Align):**
$$
P_{rotated} = V \cdot P_{scaled} = 
\begin{bmatrix}
0.78 & -0.58 & 0.23 \\
0.57 & 0.81 & 0.12 \\
0.25 & -0.05 & -0.96
\end{bmatrix} 
\begin{bmatrix} 0 \\ 0 \\ 1.24 \end{bmatrix} 
\approx \begin{bmatrix} 0.28 \\ 0.15 \\ -1.19 \end{bmatrix}
$$

**3. Translation (Shift):**
$$
P_{final} = P_{rotated} + \mu = 
\begin{bmatrix} 0.28 \\ 0.15 \\ -1.19 \end{bmatrix} + \begin{bmatrix} 5 \\ 0 \\ 2 \end{bmatrix} 
= \begin{bmatrix} 5.28 \\ 0.15 \\ 0.81 \end{bmatrix}
$$

### Conclusion
The point that started at $(0, 0, 1)$ on the generic sphere is plotted at $(5.28, 0.15, 0.81)$ in the 3D scene. This point lies on the surface of the 2-sigma boundary.

---

# Code Explanation WalkThrough

## 1.1 Eigen Decomposition

```python
    eig_vals, eig_vecs = np.linalg.eigh(covariance)
```

Math: 
Performs $\Sigma v = \lambda v$.

- ``np.linalg.eigh``: We use eigh (Hermitian) instead of eig because covariance matrices are symmetric. It is faster and more stable.
- ``eig_vals``: The magnitude of spread (Variance) along principal 
- ``axes.eig_vecs``: The direction of the spread (Rotation matrix).

---
## 1.2. Generate Unit Sphere (Higher resolution)

```python
u = np.linspace(0, 2 * np.pi, resolution)
v = np.linspace(0, np.pi, resolution)
```

Grid Setup: Creates linear arrays for spherical angles.
- ``u (Azimuth)``: 0 to 360 degrees ($2\pi$).
- ``v (Polar)``: 0 to 180 degrees ($\pi$).

## 1.3. Spherical to Cartesian: 

```python
x = np.outer(np.cos(u), np.sin(v))
y = np.outer(np.sin(u), np.sin(v))
z = np.outer(np.ones_like(u), np.cos(v))
```

**Converts angles to X, Y, Z coordinates**.
- Uses the parametric equation of a sphere: $x = \cos(u)\sin(v)$, etc.
- ``np.outer``: Computes the outer product to create a grid (mesh) of coordinates.


## 1.4. Transform Sphere to Ellipsoid

```python
sphere_coords = np.stack((x.flatten(), y.flatten(), z.flatten()))
```

**Flattening:**

- ``flatten()``: Converts the 2D grids (e.g., 100x100) into 1D arrays (10000 points).

- ``stack``: Stacks them into a (3, N) matrix. This allows us to apply matrix multiplication to all 10,000 points at once.

## 1.5. Scale by standard deviation * sigma

```python
scale_matrix = np.diag(np.sqrt(eig_vals)) * std_devs
```

Scaling Matrix ($S$):

- ``np.sqrt(eig_vals)``: Converts Variance ($\sigma^2$) to Standard Deviation ($\sigma$).
- ``np.diag``: Creates a diagonal matrix with these values.
- ``*std_devs``: Scales the size (e.g., 2-sigma covers ~95%).



## 1.6. Apply Scaling:

```python
scaled_coords = scale_matrix @ sphere_coords
```

- ``@``: Matrix multiplication.

- Stretches the sphere along X, Y, Z axes.

## 1.7. Apply Rotation:

```python
rotated_coords = eig_vecs @ scaled_coords
```

- Multiplies by the Eigenvector matrix.

- Rotates the stretched ellipsoid to align with the data's actual orientation.

## 1.8. Translate (Shift)

```python
shifted_coords = rotated_coords + mean.reshape(-1, 1)
```

**Apply Translation:**

- ``mean.reshape(-1, 1)``: Turns the mean shape (3,) into (3, 1) for broadcasting.
- Adds the mean to every point, shifting the ellipsoid from $(0,0,0)$ to the data center.



# 1.9. Reshape for Plotly

```python
    x_surf = shifted_coords[0, :].reshape(x.shape)
    y_surf = shifted_coords[1, :].reshape(y.shape)
    z_surf = shifted_coords[2, :].reshape(z.shape)
    
    return x_surf, y_surf, z_surf, eig_vals, eig_vecs
```

**Reshaping:**

- Converts the flat (1, N) arrays back into (100, 100) grids required by Plotly's go.Surface.

In [2]:
import numpy as np
import plotly.graph_objects as go

def get_ellipsoid_coordinates(covariance, mean, std_devs=2.0, resolution=100):
    """
    Calculates ellipsoid surface coordinates and returns eigen info for plotting.
    """
    # 1. Eigen Decomposition
    eig_vals, eig_vecs = np.linalg.eigh(covariance)
    
    # 2. Generate Unit Sphere (Higher resolution)
    u = np.linspace(0, 2 * np.pi, resolution)
    v = np.linspace(0, np.pi, resolution)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones_like(u), np.cos(v))
    
    # 3. Transform Sphere to Ellipsoid
    sphere_coords = np.stack((x.flatten(), y.flatten(), z.flatten()))
    
    # Scale by standard deviation * sigma
    scale_matrix = np.diag(np.sqrt(eig_vals)) * std_devs
    scaled_coords = scale_matrix @ sphere_coords
    
    # Rotate
    rotated_coords = eig_vecs @ scaled_coords
    
    # Translate (Shift)
    shifted_coords = rotated_coords + mean.reshape(-1, 1)
    
    # Reshape for Plotly
    x_surf = shifted_coords[0, :].reshape(x.shape)
    y_surf = shifted_coords[1, :].reshape(y.shape)
    z_surf = shifted_coords[2, :].reshape(z.shape)
    
    return x_surf, y_surf, z_surf, eig_vals, eig_vecs

# 2. Generate Synthetic Data
We generate 500 random 3D points based on a "Ground Truth" mean and covariance. 
In a real-world scenario, you would skip this step and load your own dataset (e.g., from a CSV or LiDAR scan).

In [3]:
np.random.seed(42)

# Ground Truth
true_mean = np.array([5, 0, 2])
true_cov = np.array([
    [3.0, 1.5, 0.5],
    [1.5, 2.0, 0.2],
    [0.5, 0.2, 0.5]
])

# Generate 1000 random points (More data points)
data = np.random.multivariate_normal(true_mean, true_cov, 1000)

## 3. Calculate Statistics:

We calculate the sample mean and sample covariance from the data.
- **Note:** We use ``rowvar=False`` because our data matrix is shaped $(N, 3)$, meaning the columns are the variables ($x, y, z$).

In [4]:
# Calculate the mean and covariance from the actual data points
calc_mean = np.mean(data, axis=0)
calc_cov = np.cov(data, rowvar=False)

print("Calculated Mean:\n", calc_mean)
print("\nCalculated Covariance:\n", calc_cov)

Calculated Mean:
 [ 4.89908733 -0.05632755  2.01107616]

Calculated Covariance:
 [[2.90794902 1.35301185 0.53078497]
 [1.35301185 1.84471813 0.17616058]
 [0.53078497 0.17616058 0.5050611 ]]


## 4. Generate Surface Data

- We call our helper function to get the X, Y, and Z grids for the ellipsoid surface. 
- We use ``std_devs=2`` (2-sigma), which theoretically encloses ``about ~74% of the points in 3D space (or ~95% in 1D)``.

In [5]:
# Constants
SIGMA = 2.0

# Get surface and eigen info
x_ell, y_ell, z_ell, eig_vals, eig_vecs = get_ellipsoid_coordinates(
    calc_cov, calc_mean, std_devs=SIGMA, resolution=100
)

## 5. Visualization

Finally, we create the interactive Plotly figure.

- ``Scatter3d``: Draws the individual points.

- ``Surface``: Draws the semi-transparent ellipsoid.

- ``aspectmode='data'``: Ensures the 3D box isn't artificially stretched into a cube, keeping the physical proportions correct.

## 5.1. Visualize the scatter plot.

```
x=data[:, 0], y=data[:, 1], z=data[:, 2],
```

Visualize X, Y and Z for the dataset.

In [6]:
fig = go.Figure()

# --- 1. Plot Data Points ---
fig.add_trace(go.Scatter3d(
    x=data[:, 0], y=data[:, 1], z=data[:, 2],
    mode='markers',
    marker=dict(size=2, color=data[:, 2], colorscale='Viridis', opacity=0.5),
    name='Data Points'
))

fig.show()

In [7]:
# --- 2. Plot Ellipsoid Surface ---
fig.add_trace(go.Surface(
    x=x_ell, y=y_ell, z=z_ell,
    opacity=0.3,
    colorscale='Blues',
    showscale=False,
    name=f'{SIGMA}-Sigma Ellipsoid'
))

fig.show()

In [8]:
# --- 3. Plot Eigenvectors (The Axes) ---
# We loop 3 times, once for each principal axis
colors = ['red', 'green', 'orange']
labels = ['Eigenvector 1 (Largest)', 'Eigenvector 2', 'Eigenvector 3 (Smallest)']

for i in range(3):
    # Calculate vector tip position
    # Tip = Mean + (Eigenvector * Sqrt(Eigenvalue) * Sigma)
    vector_length = np.sqrt(eig_vals[i]) * SIGMA
    vector_direction = eig_vecs[:, i]
    
    start = calc_mean
    end = calc_mean + (vector_direction * vector_length)
    
    # Add a Line trace for the vector shaft
    fig.add_trace(go.Scatter3d(
        x=[start[0], end[0]], 
        y=[start[1], end[1]], 
        z=[start[2], end[2]],
        mode='lines+text',
        line=dict(color=colors[i], width=5),
        text=["", labels[i]], # Label only the tip
        textposition="top center",
        name=labels[i]
    ))
    
    # Add a Cone/Marker at the tip to make it look like an arrow
    fig.add_trace(go.Scatter3d(
        x=[end[0]], y=[end[1]], z=[end[2]],
        mode='markers',
        marker=dict(size=5, color=colors[i], symbol='diamond'),
        showlegend=False
    ))

# --- Layout Updates ---
fig.update_layout(
    title='3D Covariance with Principal Axes (Eigenvectors)',
    scene=dict(
        xaxis_title='X', yaxis_title='Y', zaxis_title='Z',
        aspectmode='data'
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

fig.show()